# ANPR Archivio storico dei comuni

Data processing che partendo dal dataset di [ANPR](https://www.anpr.interno.it/portale/) contenente l'Archivio storico dei comuni (02-04-2019) - xls predispone e arricchisce il dataset in modo che possa essere importato in DAF e  interoperabile con il dataset rilasciato da ISTAT.

Il dataset verrà poi utilizzato come dataset master anche per l'arricchimento delle informazioni ad oggi contenute in IPA oltre ad essere collegato a diverse grafi come ISPRA, DBPedia...

Su DAF è presente il dataset originario creato da Alberto
https://dataportal.daf.teamdigitale.it/#/private/dataset/anpr_comuni_italiani

Sul dataset pubblicato da ANPR abbiamo aperto degli issue richiedendo che il dataset venga propriamente pubblicato seguendo le linee guida

- https://github.com/italia/anpr/issues/1252
- https://github.com/italia/anpr/issues/1253
 

Operazioni da fare pre processing:

- scraper dataset: non c'è una permament uri
- utilizzo encoding corretto
- attenzione a campi null
- il codice istat deve essere letto come string 
- campo IDPROVINCIA deve essere su tre char con padding a 0 per poter fare join con ISTAT
- formattazione campi data
- formattazione campi nome comune per denominazione in altra lingua 
- formattazione campi nome comune it e denominazione translitterata

Post processing:
- script SQL per estrarre i blocchi di comuni con denominazione in DE e FR per interlinking con Giorgia


In [166]:
import pandas as pd
import numpy as np

In [167]:
# default data per comuni validi
data_cessazione = '9999-12-31'

In [168]:
#fare attenzione a codice istat che deve essere letto come stringa per non perdere gli 0 in testa
#na_filter=False per non inserire NaN ma "" dove i campi sono vuoti

In [169]:
anpr = pd.read_excel("./data/tabella 3 archivio comuni 20190402.xlsx",  dtype={'CODISTAT': str}, na_filter=False)

In [170]:
anpr.dtypes

ID                          int64
DATAISTITUZIONE            object
DATACESSAZIONE             object
CODISTAT                   object
CODCATASTALE               object
DENOMINAZIONE_IT           object
DENOMTRASLITTERATA         object
ALTRADENOMINAZIONE         object
ALTRADENOMTRASLITTERATA    object
IDPROVINCIA                 int64
IDREGIONE                  object
IDPREFETTURA               object
STATO                      object
SIGLAPROVINCIA             object
FONTE                      object
DATAULTIMOAGG              object
COD_DENOM                  object
dtype: object

In [171]:
# il campo provincia è numerico ma su dataset è presentato in formato alfanumerico quindi bisogna trasformarlo
# in string e fare un padding left

### Check campi null

In [172]:
anpr.isnull().sum()

ID                         0
DATAISTITUZIONE            0
DATACESSAZIONE             0
CODISTAT                   0
CODCATASTALE               0
DENOMINAZIONE_IT           0
DENOMTRASLITTERATA         0
ALTRADENOMINAZIONE         0
ALTRADENOMTRASLITTERATA    0
IDPROVINCIA                0
IDREGIONE                  0
IDPREFETTURA               0
STATO                      0
SIGLAPROVINCIA             0
FONTE                      0
DATAULTIMOAGG              0
COD_DENOM                  0
dtype: int64

In [173]:
anpr.head(100)

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM
0,12560,19/11/1866,13/11/1924,028001,A001,ABANO,ABANO,,,28,05,,C,PD,,17/06/2016,028500
1,1,14/11/1924,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,,,28,05,PD,A,PD,,17/06/2016,
2,3,01/04/1864,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,1,01,,C,TO,,11/11/2017,
3,5,17/03/1861,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,15,03,,C,MI,,17/06/2016,
4,4,16/04/1992,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,98,03,LO,A,LO,,17/06/2016,
5,6,06/03/1928,15/04/1992,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,13,03,,C,CO,,17/06/2016,
6,7,16/04/1992,31/12/9999,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,97,03,LC,A,LC,,17/06/2016,
7,8,22/07/1867,31/12/9999,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,52,09,SI,A,SI,,17/06/2016,
8,2,23/02/1863,06/03/1928,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,13,03,,C,CO,,17/06/2016,
9,11,17/03/1861,27/07/1927,092001,A007,ABBASANTA,ABBASANTA,,,92,20,,C,CA,,17/06/2016,


## Adattamento campo IDPROVINCIA 

Leggerlo come striga e fare padding con 0 su tre char

In [174]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(str)

In [175]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(lambda x: x.zfill(3))

In [177]:
anpr['IDPROVINCIA'].head(10)

0    028
1    028
2    001
3    015
4    098
5    013
6    097
7    052
8    013
9    092
Name: IDPROVINCIA, dtype: object

## Adattamento formato campi data

I campi data, che al momento non sono gestiti sul form di ingestion del DAF, vanno trasformati usando il "-" come separatore.

Da fare per il dataset nuovo per il cui non sarà più necessario fare vista.

In [154]:
anpr['DATAULTIMOAGG'] = pd.to_datetime(anpr['DATAULTIMOAGG'])
anpr['DATAULTIMOAGG'] = anpr['DATAULTIMOAGG'].dt.strftime('%Y-%m-%d')

In [155]:
#su data ultimo agg bisogna ignorare errori
anpr['DATAISTITUZIONE'] = pd.to_datetime(anpr['DATAISTITUZIONE'])
anpr['DATAISTITUZIONE'] = anpr['DATAISTITUZIONE'].dt.strftime('%Y-%m-%d')

In [156]:
anpr['DATACESSAZIONE'] = pd.to_datetime(anpr['DATACESSAZIONE'], errors='coerce')

In [135]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].dt.strftime('%Y-%m-%d')

In [157]:
#NON FATTO
#anpr['DATACESSAZIONE'].date.dt.strftime('%Y-%m-%d').replace('NaT', data_cessazione)

In [158]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].replace('NaT', data_cessazione)

In [182]:
anpr[['DATAULTIMOAGG','DATAISTITUZIONE','DATACESSAZIONE']].tail(20)

,DATAULTIMOAGG,DATAISTITUZIONE,DATACESSAZIONE
19283,17/06/2016,17/03/1861,11/01/1927
19284,17/06/2016,12/01/1927,03/06/1929
19285,17/06/2016,04/06/1929,31/12/9999
19286,17/06/2016,16/10/1920,04/02/1923
19287,11/11/2017,05/02/1923,23/02/1928
19288,17/06/2016,17/05/1952,31/12/2015
19289,17/06/2016,19/11/1866,02/06/1943
19290,17/06/2016,03/06/1943,31/12/9999
19291,17/06/2016,19/11/1866,25/09/1932
19292,17/06/2016,15/11/1946,03/03/1959


## Adattamento 
###  DENOMINAZIONE_IT e DENOMTRASLITTERATA

1) Tutte le denominazioni in italiano **translitterate** le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole
così diventa confrontabile con i comuni di IPA

2) Tutte le denominazioni in italiano (non translitterate) le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole. 

In presenza di doppia parola (es ABANO TERME) dobbiamo farla diventare LetteraMaiuscolaLettereminuscole_LetteraMaiuscolalettereminuscole  
queste due cose sono fondamentali anche per diversi join sui dataset IPA e per fare intelinking con DBPedia.

In [183]:
anpr['result'] = np.where(anpr['DENOMINAZIONE_IT'] == anpr['DENOMTRASLITTERATA'], 'no change', 'changed')
print (anpr.result.value_counts())

no change    18890
changed        413
Name: result, dtype: int64


In [184]:
anpr[anpr['result']=='changed']

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,result
93,17704,17/03/1861,22/08/1929,087006,A056,ADERNÒ,ADERNO',,,087,19,,C,CT,,17/06/2016,087500,changed
126,17720,17/03/1861,11/01/1927,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,17/06/2016,,changed
127,22003,12/01/1927,14/06/1929,007805,A074,AGLIÈ,AGLIE',,,007,02,,C,AO,,17/06/2016,,changed
128,22004,15/06/1929,04/10/1945,007805,A074,AGLIÈ,AGLIE',,,007,02,,C,AO,,17/06/2016,,changed
129,17718,05/10/1945,09/01/1947,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,17/06/2016,,changed
130,17719,10/01/1947,23/01/1947,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,17/06/2016,,changed
131,82,24/01/1947,31/12/9999,001001,A074,AGLIÈ,AGLIE',,,001,01,TO,A,TO,,17/06/2016,,changed
210,130,14/01/1864,31/12/2005,090002,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,090,20,,C,SS,,17/06/2016,,changed
211,129,01/01/2006,27/04/2016,104003,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,104,20,,C,OT,,11/11/2017,,changed
212,22823,28/04/2016,31/12/9999,090002,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,090,20,SS,A,SS,,11/11/2017,,changed


In [185]:
anpr["Denominazione_it_linking"] = anpr.DENOMINAZIONE_IT.str.title()
anpr["Denominazione_it_linking"] = anpr.Denominazione_it_linking.str.replace(" ", "_")

In [186]:
anpr["DENOMTRASLITTERATA_IPA"]=anpr.DENOMTRASLITTERATA.str.title()

In [187]:
anpr

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,result,Denominazione_it_linking,DENOMTRASLITTERATA_IPA
0,12560,19/11/1866,13/11/1924,028001,A001,ABANO,ABANO,,,028,05,,C,PD,,17/06/2016,028500,no change,Abano,Abano
1,1,14/11/1924,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,,,028,05,PD,A,PD,,17/06/2016,,no change,Abano_Terme,Abano Terme
2,3,01/04/1864,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,001,01,,C,TO,,11/11/2017,,no change,Abbadia_Alpina,Abbadia Alpina
3,5,17/03/1861,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,015,03,,C,MI,,17/06/2016,,no change,Abbadia_Cerreto,Abbadia Cerreto
4,4,16/04/1992,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,098,03,LO,A,LO,,17/06/2016,,no change,Abbadia_Cerreto,Abbadia Cerreto
5,6,06/03/1928,15/04/1992,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,013,03,,C,CO,,17/06/2016,,no change,Abbadia_Lariana,Abbadia Lariana
6,7,16/04/1992,31/12/9999,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,097,03,LC,A,LC,,17/06/2016,,no change,Abbadia_Lariana,Abbadia Lariana
7,8,22/07/1867,31/12/9999,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,052,09,SI,A,SI,,17/06/2016,,no change,Abbadia_San_Salvatore,Abbadia San Salvatore
8,2,23/02/1863,06/03/1928,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,013,03,,C,CO,,17/06/2016,,no change,Abbadia_Sopra_Adda,Abbadia Sopra Adda
9,11,17/03/1861,27/07/1927,092001,A007,ABBASANTA,ABBASANTA,,,092,20,,C,CA,,17/06/2016,,no change,Abbasanta,Abbasanta


In [188]:
anpr.DATACESSAZIONE.value_counts()

31/12/9999    7915
11/01/1927    1632
04/02/1923     589
15/04/1992     368
07/02/1923     256
05/04/1968     187
19/08/1974     177
27/04/2016     155
26/04/1923     142
02/03/1970     135
31/12/2005     123
15/09/1947     118
07/04/1935     105
31/08/1945      79
21/12/1923      78
22/05/1992      77
18/03/1923      68
31/12/2015      68
03/09/1998      67
04/10/1945      63
11/05/1928      56
29/06/2009      50
12/05/1928      46
05/09/1928      45
31/12/2018      45
25/01/1928      43
12/03/1928      40
11/07/2009      40
29/01/1946      39
20/02/1928      38
              ... 
02/09/1938       1
17/08/1959       1
08/10/1959       1
31/07/1950       1
24/01/1948       1
10/02/1907       1
22/03/1975       1
21/07/1957       1
17/11/1951       1
09/04/1893       1
10/06/1917       1
27/10/1970       1
20/01/1948       1
07/10/1956       1
06/12/1934       1
21/06/1878       1
30/06/1947       1
30/04/1991       1
21/10/1999       1
24/12/1946       1
18/03/1925       1
07/02/1962  

## Salvare dataset finale con separatore e senza index

In [165]:
anpr.to_csv("./data/output.csv", index=False, sep=";")